# Grasp annotation example

In [ ]:
from pathlib import Path
import cv2
from airo_camera_toolkit.point_clouds.conversions import open3d_to_point_cloud, point_cloud_to_open3d
from airo_camera_toolkit.point_clouds.operations import filter_point_cloud
from cloth_tools.dataset.format import load_competition_input_sample
import matplotlib.pyplot as plt
from cloth_tools.dataset.format import load_competition_input_sample
from airo_camera_toolkit.utils.image_converter import ImageConverter
import open3d as o3d

data_dir = Path("../data")
dataset_dir = data_dir / "dataset_0000"

In [ ]:
sample = load_competition_input_sample(dataset_dir, sample_index=0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(sample.image_left)

In [ ]:
confidence_map = sample.confidence_map
point_cloud_in_camera = sample.point_cloud

# Transform point cloud to world frame
X_W_C = sample.camera_pose_in_world  # X_LCB_C (camera pose in the left-arm base frame)
pcd_in_camera = point_cloud_to_open3d(point_cloud_in_camera)  # X_C_PC, need X_W_C
pcd = pcd_in_camera.transform(X_W_C)  # transform to world frame
point_cloud = open3d_to_point_cloud(pcd)

# Filter outs point with low depth confidence (i.e. with high value in the confidence map)
confidence_threshold = 1.0
confidence_mask = (confidence_map <= confidence_threshold).reshape(-1)  # Threshold and flatten
point_cloud_filtered = filter_point_cloud(point_cloud, confidence_mask)
pcd_filtered = point_cloud_to_open3d(point_cloud_filtered)
pcd_filtered.point.positions.dtype, pcd_filtered.point.colors.dtype

In [ ]:
from cloth_tools.annotation.grasp_annotation import top_down_camera_pose

virtual_camera_pose = top_down_camera_pose(height=1.5)

In [ ]:
from cloth_tools.visualization.open3d import open3d_camera

color_frontal_rgb = (1, 1, 0)
color_topdown_rgb = (0, 1, 1)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)

# Visualize the cameras
resolution = sample.camera_resolution
intrinsics = sample.camera_intrinsics

X_W_VC = virtual_camera_pose
camera_frontal_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
camera_frontal_frame.transform(X_W_C)
camera_top_down_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
camera_top_down_frame.transform(X_W_VC)
camera_frontal_lines = open3d_camera(X_W_C, intrinsics, resolution, color_frontal_rgb, scale=0.2)
camera_topdown_lines = open3d_camera(X_W_VC, intrinsics, resolution, color_topdown_rgb, scale=0.2)

o3d.visualization.draw_geometries(
    [
        pcd_filtered.to_legacy(),
        world_frame,
        camera_frontal_frame,
        camera_top_down_frame,
        camera_frontal_lines,
        camera_topdown_lines,
    ]
)

In [ ]:
from cloth_tools.annotation.grasp_annotation import project_point_cloud_to_image

image_topdown = project_point_cloud_to_image(
    point_cloud_filtered, X_W_VC, intrinsics, resolution, background_color=(90, 90, 90)
)

# Matplotlib seems to do some anti-aliasing, which makes the image look better than in opencv (without blurring)
plt.figure(figsize=(12, 6))
plt.imshow(ImageConverter.from_opencv_format(image_topdown).image_in_numpy_int_format)
plt.show()

In [ ]:
# Some experiments with blurring to make the image look better in opencv
width = 800
height = 400
background_color = (120, 120, 120)

image_topdown = project_point_cloud_to_image(point_cloud_filtered, X_W_VC, intrinsics, resolution, background_color)

window_name_original = "Original"
cv2.namedWindow(window_name_original, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name_original, width, height)
cv2.moveWindow(window_name_original, 0, 0)
cv2.imshow(window_name_original, image_topdown)

window_name_median_blur = "Median Blur"
cv2.namedWindow(window_name_median_blur, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name_median_blur, width, height)
cv2.moveWindow(window_name_median_blur, width, 0)
image_median_blur = cv2.medianBlur(image_topdown, 3)
cv2.imshow(window_name_median_blur, image_median_blur)

window_name_blur = "Blur"
cv2.namedWindow(window_name_blur, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name_blur, width, height)
cv2.moveWindow(window_name_blur, 0, height)
image_blur = cv2.blur(image_topdown, (3, 3))
cv2.imshow(window_name_blur, image_blur)

window_name_combined = "Blur + Median"
cv2.namedWindow(window_name_combined, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name_combined, width, height)
cv2.moveWindow(window_name_combined, width, height)
image_blur_median = cv2.medianBlur(image_blur, 3)
cv2.imshow(window_name_combined, image_blur_median)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from cloth_tools.annotation.grasp_annotation import get_manual_grasp_annotation

grasp_pose = get_manual_grasp_annotation(
    sample.image_left, sample.depth_map, point_cloud_filtered, X_W_C, intrinsics, log_to_rerun=True
)

In [ ]:
grasp_pose